In [ ]:
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import tensorflow 
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load in hdf5 files
test = h5py.File("/content/drive/MyDrive/VDISC_test.hdf5",'r')
train = h5py.File("/content/drive/MyDrive/VDISC_train.hdf5",'r')
validate = h5py.File("/content/drive/MyDrive/VDISC_validate.hdf5",'r')

In [ ]:
test.visit(print)
mydf = pd.DataFrame(list(test['functionSource']))
mydf['CWE-119']=list(test['CWE-119']); mydf['CWE-120']=list(test['CWE-120']); mydf['CWE-469']=list(test['CWE-469']); mydf['CWE-476']=list(test['CWE-476']); mydf['CWE-other']=list(test['CWE-other']) 
mydf.rename(columns={0:'functionSource'},inplace=True)
mydf.iloc[0:5,0:]
mydf.to_pickle("VDISC_test.pickle")

CWE-119
CWE-120
CWE-469
CWE-476
CWE-other
functionSource


In [ ]:
train.visit(print)
mydf = pd.DataFrame(list(train['functionSource']))
mydf['CWE-119']=list(train['CWE-119']); mydf['CWE-120']=list(train['CWE-120']); mydf['CWE-469']=list(train['CWE-469']); mydf['CWE-476']=list(train['CWE-476']); mydf['CWE-other']=list(train['CWE-other']) 
mydf.rename(columns={0:'functionSource'},inplace=True)
mydf.iloc[0:5,0:]
mydf.to_pickle("VDISC_train.pickle")

CWE-119
CWE-120
CWE-469
CWE-476
CWE-other
functionSource


In [ ]:
validate.visit(print)
mydf = pd.DataFrame(list(validate['functionSource']))
mydf['CWE-119']=list(validate['CWE-119']); mydf['CWE-120']=list(validate['CWE-120']); mydf['CWE-469']=list(validate['CWE-469']); mydf['CWE-476']=list(validate['CWE-476']); mydf['CWE-other']=list(validate['CWE-other']) 
mydf.rename(columns={0:'functionSource'},inplace=True)
mydf.iloc[0:5,0:]
mydf.to_pickle("VDISC_validate.pickle")

CWE-119
CWE-120
CWE-469
CWE-476
CWE-other
functionSource


In [ ]:
# Set the global value
WORDS_SIZE=10000
INPUT_SIZE=500
NUM_CLASSES=2
MODEL_NUM=0
EPOCHS=10
BATCH_SIZE=128

In [ ]:
test = pd.read_pickle("/content/drive/MyDrive/VDISC_test.pickle")
train = pd.read_pickle("/content/drive/MyDrive/VDISC_train.pickle")
validate = pd.read_pickle("/content/drive/MyDrive/VDISC_validate.pickle")

In [ ]:
# Overview of the datasets
#pd.value_counts(train.iloc[:,1])

In [ ]:
one = train[train.iloc[:,1]==1].index.values.astype(int)
zero = train[train.iloc[:,1]==0].index.values.astype(int)

In [ ]:
# map True and False values to 1 and 0
for dataset in [train,validate,test]:
    for col in range(1,6):
        dataset.iloc[:,col] = dataset.iloc[:,col].map({False: 0, True: 1})

# Create source code data for tokenization
x_vals = train['functionSource'].values
np.append(x_vals, validate['functionSource'].values)
np.append(x_vals, test['functionSource'].values)
print(x_vals)

[b'clear_area(int startx, int starty, int xsize, int ysize)\n{\n  int x;\n\n  TRACE_LOG("Clearing area %d,%d / %d,%d\\n", startx, starty, xsize, ysize);\n\n  while (ysize > 0)\n  {\n    x = xsize;\n    while (x > 0)\n    {\n      mvaddch(starty + ysize - 2, startx + x - 2, \' \');\n      x--;\n    }\n    ysize--;\n  }\n}'
 b'ReconstructDuList(Statement* head)\n{\n    Statement* spt;\n\n    for (spt = head; spt != NULL; spt = spt->next) {\n\tdelete_def_use_list(spt->use_var_list);\n\tdelete_def_use_list(spt->def_var_list);\n\tdelete_def_use_list(spt->use_array_list);\n\tdelete_def_use_list(spt->def_array_list);\n\tspt->def_var_list = NULL;\n\tspt->use_var_list = NULL;\n\tspt->def_array_list = NULL;\n\tspt->use_array_list = NULL;\n    }\n    def_use_statement(head);\n}'
 b'free_speaker(void)\n{\n   if(Lengths)\n       free(Lengths);\n\n   if(!audio2fast && commento)\n       fclose(commento);\n\n\n   frase = NON_DECISA;\n   game_status = S_NON_INIZIATO;\n\n   fondolen = sound[FONDO]->Leng

In [ ]:
train.head()

,functionSource,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other
0,"b'clear_area(int startx, int starty, int xsize...",0,0,0,0,0
1,b'ReconstructDuList(Statement* head)\n{\n S...,0,0,0,0,0
2,b'free_speaker(void)\n{\n if(Lengths)\n ...,0,0,0,0,0
3,b'mlx4_register_device(struct mlx4_dev *dev)\n...,0,0,0,0,0
4,"b'Parse_Env_Var(void)\n{\n char *p = getenv(""...",1,1,0,0,1


In [ ]:
# Tokenizer with word-level
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
print(x_vals)
tokenizer.fit_on_texts([str(i) for i in x_vals[:10000]])
#del(x_vals)
print('Number of tokens: ',len(tokenizer.word_counts))

[b'clear_area(int startx, int starty, int xsize, int ysize)\n{\n  int x;\n\n  TRACE_LOG("Clearing area %d,%d / %d,%d\\n", startx, starty, xsize, ysize);\n\n  while (ysize > 0)\n  {\n    x = xsize;\n    while (x > 0)\n    {\n      mvaddch(starty + ysize - 2, startx + x - 2, \' \');\n      x--;\n    }\n    ysize--;\n  }\n}'
 b'ReconstructDuList(Statement* head)\n{\n    Statement* spt;\n\n    for (spt = head; spt != NULL; spt = spt->next) {\n\tdelete_def_use_list(spt->use_var_list);\n\tdelete_def_use_list(spt->def_var_list);\n\tdelete_def_use_list(spt->use_array_list);\n\tdelete_def_use_list(spt->def_array_list);\n\tspt->def_var_list = NULL;\n\tspt->use_var_list = NULL;\n\tspt->def_array_list = NULL;\n\tspt->use_array_list = NULL;\n    }\n    def_use_statement(head);\n}'
 b'free_speaker(void)\n{\n   if(Lengths)\n       free(Lengths);\n\n   if(!audio2fast && commento)\n       fclose(commento);\n\n\n   frase = NON_DECISA;\n   game_status = S_NON_INIZIATO;\n\n   fondolen = sound[FONDO]->Leng

In [ ]:
# reduce words
tokenizer.num_words = WORDS_SIZE

# top 10 words in file
sorted(tokenizer.word_counts.items(), key=lambda x:x[1], reverse=True)[0:10]

[('n', 288171),
 ('t', 129874),
 ('0', 21020),
 ('if', 18372),
 ("'", 14338),
 ('i', 13810),
 ('1', 11741),
 ('tif', 11714),
 ('return', 9614),
 ('null', 9241)]

In [ ]:
## create matrix after tokenizing training data
list_data= train['functionSource'].values
tokenized_train = tokenizer.texts_to_sequences([str(i) for i in list_data])
x_train = tf.keras.preprocessing.sequence.pad_sequences(tokenized_train, 
                                  maxlen=INPUT_SIZE,
                                  padding='post')
x_train = x_train.astype(np.int64)
print(x_train.shape)

## create matrix after tokenizing testing data
list_data= test['functionSource'].values
tokenized_test = tokenizer.texts_to_sequences([str(i) for i in list_data])
x_test = tf.keras.preprocessing.sequence.pad_sequences(tokenized_test, 
                                 maxlen=INPUT_SIZE,
                                 padding='post')
x_test = x_test.astype(np.int64)
print(x_test.shape)

## create matrix after tokenizing validation data
list_data= validate['functionSource'].values
tokenized_validate = tokenizer.texts_to_sequences([str(i) for i in list_data])
x_validate = tf.keras.preprocessing.sequence.pad_sequences(tokenized_validate, 
                                 maxlen=INPUT_SIZE,
                                 padding='post')
x_validate = x_validate.astype(np.int64)
print(x_validate.shape)

(1019471, 500)
(127419, 500)
(127476, 500)


In [ ]:
# example of the data
print('Train', pd.value_counts(train.iloc[:,1]))
print('\nTest', pd.value_counts(test.iloc[:,1]))
print('\nValidate', pd.value_counts(validate.iloc[:,1]))

Train 0    1000185
1      19286
Name: CWE-119, dtype: int64

Test 0    124967
1      2452
Name: CWE-119, dtype: int64

Validate 0    125057
1      2419
Name: CWE-119, dtype: int64


In [ ]:
# Example data
test.iloc[0:5,1:6]

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [ ]:
y_train=[]
y_test=[]
y_validate=[]

for col in range(1,6):
    y_train.append(tf.keras.utils.to_categorical(train.iloc[:,col], num_classes=NUM_CLASSES).astype(np.int64))
    y_test.append(tf.keras.utils.to_categorical(test.iloc[:,col], num_classes=NUM_CLASSES).astype(np.int64))
    y_validate.append(tf.keras.utils.to_categorical(validate.iloc[:,col], num_classes=NUM_CLASSES).astype(np.int64))



In [ ]:
# Example data
pd.value_counts(y_test[0][:,1])

0    124967
1      2452
dtype: int64

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution1D, Dropout, GlobalAveragePooling1D
#from keras.optimizers import SGD
from keras.initializers import random_uniform

import pandas as pd

In [ ]:
#hyperparameters
#input_dimension = 226
learning_rate = 0.005
beta1 = 0.9
beta2=0.999
epsilon=1
decay = 0.0
amsgrad = False

dropout_rate = 0.5


In [ ]:
#initialize random weights
model_weights = np.random.normal(size=(WORDS_SIZE, 13),scale=0.01)

# implement sequential CNN model
model = tf.keras.Sequential(name="CNN")
model.add(tf.keras.layers.Embedding(input_dim = WORDS_SIZE,
                                    output_dim = 13,
                                    weights=[model_weights],
                                    input_length = INPUT_SIZE))

model.add(tf.keras.layers.Convolution1D(filters=512, kernel_size=(9), padding='same', activation='relu'))
#model.add(tf.keras.layers.Convolution1D(filters=256, kernel_size=(9), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=4))
model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
#model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#implement adam optimizer
adam_optimizer = tf.keras.optimizers.Adam(learning_rate, beta1, beta2, epsilon, decay, amsgrad)

#compile model with optimizer and metrics
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print("CNN model built: ")
model.summary()

CNN model built: 
Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 13)           130000    
                                                                 
 conv1d (Conv1D)             (None, 500, 512)          60416     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 125, 512)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 125, 512)          0         
                                                                 
 flatten (Flatten)           (None, 64000)             0         
                                                                 
 dense (Dense)               (None, 64)                4096064   
                                             

In [ ]:
mld = 'model/model-ALL-{epoch:02d}.hdf5'

## Create best model callback
model_callback = tf.keras.callbacks.ModelCheckpoint(filepath=mld, 
                                         monitor="val_loss",
                                         save_best_only=True, 
                                         mode='auto', 
                                         save_freq='epoch', 
                                         verbose=1)

In [ ]:

ceiling = 38572


class_weights = {0: 1., 1: 5.}

model.fit(x = x_train[[*one,*zero[0:ceiling]],:],
          y = train.iloc[[*one,*zero[0:ceiling]],1].to_numpy(),
          validation_data = (x_validate, validate.iloc[:,1].to_numpy()),
          epochs = 30,
          batch_size = 16,
          verbose =2,
          class_weight= class_weights,
          callbacks = [model_callback])


Train on 57858 samples, validate on 127476 samples
Epoch 1/30


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



Epoch 1: val_loss improved from inf to 1.01734, saving model to model/model-ALL-01.hdf5
57858/57858 - 292s - loss: 1.3118 - accuracy: 0.4091 - val_loss: 1.0173 - val_accuracy: 0.2991 - 292s/epoch - 5ms/sample
Epoch 2/30

Epoch 2: val_loss improved from 1.01734 to 0.92068, saving model to model/model-ALL-02.hdf5
57858/57858 - 324s - loss: 1.0299 - accuracy: 0.6435 - val_loss: 0.9207 - val_accuracy: 0.6008 - 324s/epoch - 6ms/sample
Epoch 3/30

Epoch 3: val_loss improved from 0.92068 to 0.68593, saving model to model/model-ALL-03.hdf5
57858/57858 - 317s - loss: 0.7875 - accuracy: 0.8059 - val_loss: 0.6859 - val_accuracy: 0.7387 - 317s/epoch - 5ms/sample
Epoch 4/30

Epoch 4: val_loss improved from 0.68593 to 0.54616, saving model to model/model-ALL-04.hdf5
57858/57858 - 294s - loss: 0.7386 - accuracy: 0.8236 - val_loss: 0.5462 - val_accuracy: 0.8118 - 294s/epoch - 5ms/sample
Epoch 5/30

Epoch 5: val_loss improved from 0.54616 to 0.45587, saving model to model/model-ALL-05.hdf5
57858/57858